In [1]:
import pydicom
import os,re
import numpy as np
import pandas as pd
import shutil
import matplotlib.pyplot as plt
%matplotlib inline
import SimpleITK as sitk
import math
import sys

import AssistVisualization as vis

from MyProcessDicom import *
from LocalDicomIO import *
from RegistrationMethod import *

import cv2

transform_path = "./data/9404954L/transform/"

In [2]:
file_path = "./data/OAIMTP/P9404954L/tracing_raw_ves_TH_0_P9404954L_U.swc"

In [3]:
def get_centerline(file_path):
    df = pd.read_csv(file_path, header=None, sep=' ',index_col=0, names=['vessel_id','x','y','z','undefined','last_id'])
    line = np.array([df['x'], df['y'], df['z']]).T
    return df, line

In [4]:
df1, line1 = get_centerline(file_path)
df1.head(20)

,vessel_id,x,y,z,undefined,last_id
1,0,207.0,293.0,4.11,8.85,0
2,0,206.0,294.0,8.23,8.88,1
3,0,205.0,295.0,12.34,8.99,2
4,0,204.0,295.0,16.46,8.83,3
5,0,203.0,295.0,20.57,8.94,4
6,0,202.0,295.0,24.69,9.06,5
7,0,201.0,296.0,28.80,9.15,6
8,0,200.0,296.0,32.91,9.06,7
9,0,199.0,296.0,37.03,9.24,8
10,0,198.0,296.0,41.14,9.32,9


In [5]:
file_path2 = "./data/OAIMTP/P9404954L/tracing_raw_ves_TH_1_P9404954L_U.swc"
df2, line2 = get_centerline(file_path2)
df2.head(20)

,vessel_id,x,y,z,undefined,last_id
1,0,217.0,305.0,4.11,8.29,0
2,0,216.0,305.0,8.23,8.76,1
3,0,215.0,305.0,12.34,8.79,2
4,0,213.0,305.0,16.46,9.20,3
5,0,212.0,305.0,20.57,9.10,4
6,0,211.0,305.0,24.69,9.15,5
7,0,211.0,305.0,28.80,9.08,6
8,0,210.0,305.0,32.91,9.15,7
9,0,208.0,306.0,37.03,9.18,8
10,0,207.0,306.0,41.14,9.30,9


In [6]:
affine = sitk.ReadTransform(transform_path+"affineTP1to0.tfm")

In [7]:
def generate_regis_with_centerline(line1, line2, transform):
    points = []
    for i in range(line1.shape[0]):
        tmp_point = transform.TransformPoint((line1[i][0], line1[i][1], i))
        points.append(tmp_point)
    ans = np.array(points)
    ans[:,2] = np.round(ans[:,2])
    dx, dy = [], []
    for i in range(ans.shape[0]):
        if int(ans[i][2]) >= 75:
            break
        dx.append(ans[i][0] - line2[int(ans[i][2])][0])
        dy.append(ans[i][1] - line2[int(ans[i][2])][1])
    dx = np.array(dx)
    dy = np.array(dy)
    displacement_image = sitk.Image([384,384,dx.shape[0]], sitk.sitkVectorFloat64)
    # The only point that has any displacement is (0,0)
    for i in range(dx.shape[0]):
        displacement = (-dx[i],-dy[i],0)
        for j in range(384):
            for k in range(384):
                displacement_image[j,k,i] = displacement
    return sitk.DisplacementFieldTransform(displacement_image)

In [50]:
displacement_field_transform = generate_regis_with_centerline(line1, line2, affine)

In [9]:
# print(displacement_field_transform)
displacement_field_transform = sitk.ReadTransform(transform_path+"displacement1.tfm")

In [51]:
file_path = "./data/9404954L/TP0"
img_array, fixed_image = read_dicom_image(file_path)
file_path = "./data/9404954L/TP1"
img_array, moving_image = read_dicom_image(file_path)
show_images(fixed_image, moving_image)

75
size = (384, 384, 75)
75
size = (384, 384, 75)


interactive(children=(IntSlider(value=37, description='fixed_image_z', max=74), IntSlider(value=37, descriptio…

In [52]:
resampled = transform_img(empty_preprocess(fixed_image), empty_preprocess(moving_image), affine)

In [53]:
show_mixed_images(empty_preprocess(fixed_image), resampled)

interactive(children=(IntSlider(value=37, description='image_z', max=74), FloatSlider(value=0.5, description='…

In [54]:
resampled2 = transform_img(empty_preprocess(fixed_image), resampled, displacement_field_transform)

In [55]:
show_mixed_images(empty_preprocess(fixed_image), resampled2)

interactive(children=(IntSlider(value=37, description='image_z', max=74), FloatSlider(value=0.5, description='…

In [111]:
sitk.WriteTransform(displacement_field_transform, transform_path+"displacement1.tfm")
write_dicom_series(resampled2, "./data/9404954L/resample", "TP8to0_new")

In [17]:
sitk.WriteImage(sitk.Cast(resample_image(resampled2), sitk.sitkInt16), "./data/9404954L/resample/TH_0_P9404954L_US108.tif")

[384 384  75] [384, 384, 75]
